# 📊 Notebook 03 : Visualisation des Ventes

Ce notebook génère les graphiques clés pour comprendre la performance des cafés.

In [11]:
# [LOGIQUE] : Configuration du backend matplotlib pour un environnement sans interface graphique (serveur/CI)
# [SYNTAXE] : 'Agg' est un backend non-interactif qui permet de sauvegarder des fichiers sans ouvrir de fenêtre
# [SENIOR] : Toujours définir le backend AVANT d'importer pyplot pour éviter les conflits de threading
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import os

# [LOGIQUE] : Création du dossier de sortie s'il n'existe pas
# [PATHS] : chemins relatifs robustes
from pathlib import Path
BASE_DIR = Path('..')
FIG_DIR = BASE_DIR / 'outputs' / 'figures'
FIG_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from pathlib import Path
DATA_PATH = BASE_DIR / 'data' / 'generated' / 'cafe_data_2024.csv'
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Fichier de données introuvable: {DATA_PATH}")
df = pd.read_csv(DATA_PATH, parse_dates=['date'])
print(df.head())
# Sanity checks
if 'est_weekend' not in df.columns:
    df['est_weekend'] = df['date'].dt.weekday >= 5
if 'ventes_total' not in df.columns:
    if 'ventes' in df.columns:
        df['ventes_total'] = df['ventes']
    else:
        raise KeyError("La colonne 'ventes_total' est manquante et aucune colonne 'ventes' alternative trouvée.")
if 'nb_clients' not in df.columns:
    df['nb_clients'] = df.get('clients', 0)
print('Data loaded and sanitized')

        date  ventes_total  nb_clients  est_weekend
0 2024-01-01        102.48           8        False
1 2024-01-02         99.65          16        False
2 2024-01-03        103.93          14        False
3 2024-01-04        108.65           6        False
4 2024-01-05        100.21          14        False


In [ ]:
# [LOGIQUE] : Courbe temporelle des ventes pour identifier les tendances saisonnières
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['ventes_total'], label='Ventes Journalières', color='#2ecc71')
plt.title('Évolution des Ventes Totales - 2024')
plt.xlabel('Date')
plt.ylabel('Ventes (€)')
plt.grid(True, alpha=0.3)
plt.savefig(str(FIG_DIR / '01_courbe_temporelle.png'))
plt.close() # [SENIOR] : Libérer la mémoire après chaque graphique

In [ ]:
# [LOGIQUE] : Comparaison Week-end vs Semaine pour valider l'hypothèse de boost de fréquentation
avg_sales = df.groupby('est_weekend')['ventes_total'].mean()
plt.figure(figsize=(8, 5))
avg_sales.plot(kind='bar', color=['#3498db', '#e74c3c'])
plt.title('Moyenne des Ventes : Semaine vs Week-end')
plt.xticks([0, 1], ['Semaine', 'Week-end'], rotation=0)
plt.ylabel('Ventes Moyennes (€)')
plt.savefig(str(FIG_DIR / '02_barres_weekend.png'))
plt.close()

In [10]:
# [LOGIQUE] : Distribution du nombre de clients pour observer la variabilité du flux
plt.figure(figsize=(8, 5))
plt.hist(df['nb_clients'], bins=20, color='#8e44ad', alpha=0.7)
plt.title('Distribution du Nombre de Clients')
plt.xlabel('Clients par jour')
plt.ylabel('Fréquence')
plt.savefig(str(FIG_DIR / '03_histogramme_clients.png'))
plt.close()

print(f"✅ 3 graphiques générés avec succès dans {FIG_DIR}")

✅ 3 graphiques générés avec succès dans outputs/figures/
